In [7]:
import torch
M=100
N = 100
num_upper_elements = N * (N + 1) // 2
upper_triangular_elements = torch.randn(M,num_upper_elements)

# 创建一个零矩阵
A = torch.zeros(M,N, N)


indices = torch.triu_indices(N,N)

# 填充上三角部分，利用广播
A[:, indices[0], indices[1]] = upper_triangular_elements

print(A)

tensor([[[-0.5878, -0.3825, -1.5335,  ...,  1.7934,  0.3629, -0.8203],
         [ 0.0000, -1.7480, -1.3370,  ...,  0.1800, -1.0036, -0.1825],
         [ 0.0000,  0.0000, -0.5698,  ..., -0.4317, -0.6889,  0.5475],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.2098,  1.1672,  0.6841],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.2985,  0.9803],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.3093]],

        [[ 1.0673,  1.6178,  1.7278,  ...,  1.2254,  0.5577, -1.6134],
         [ 0.0000, -1.3760, -1.8791,  ..., -2.2858, -0.6441, -1.3165],
         [ 0.0000,  0.0000,  3.0263,  ...,  0.6544, -0.0651,  0.1031],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ..., -0.2165, -0.0347,  0.3096],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.7430,  0.4925],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000, -0.7092]],

        [[ 0.8401, -0.8062, -0.9149,  ...,  0.4549, -0.9444, -0.0154],
         [ 0.0000, -1.0263, -1.4823,  ..., -0

In [24]:
import random
import torch
import sys
import numpy as np
import math
import matplotlib.pyplot as plt
from timeit import default_timer
from scipy.io import loadmat
import yaml
import gc

sys.path.append("../")


from models import  newHGkNN_train, compute_2dFourier_bases, compute_2dpca_bases, compute_2dFourier_cbases, count_params

from models.myGkNN11 import newHGkNN

torch.set_printoptions(precision=16)


torch.manual_seed(0)
np.random.seed(0)


###################################
# load configs
###################################
with open('config.yml', 'r', encoding='utf-8') as f:
    config = yaml.full_load(f)

config = config["Darcy_HGkNN"]
config = dict(config)
config_data, config_model, config_train = (
    config["data"],
    config["model"],
    config["train"],
)
downsample_ratio = config_data["downsample_ratio"]
L = config_data["L"]
n_train = config_data["n_train"]
n_test = config_data["n_test"]
device = torch.device(config["train"]["device"])


###################################
# load data
###################################
# data_path = "../data/darcy_2d/piececonst_r421_N1024_smooth1"
# data1 = loadmat(data_path)
# coeff1 = data1["coeff"]
# sol1 = data1["sol"]
# del data1
# data_path = "../data/darcy_2d/piececonst_r421_N1024_smooth2"
# data2 = loadmat(data_path)
# coeff2 = data2["coeff"][:300,:,:]
# sol2 = data2["sol"][:300,:,:]
# del data2
# gc.collect()

# data_in = np.vstack((coeff1, coeff2))  # shape: 2048,421,421
# data_out = np.vstack((sol1, sol2))     # shape: 2048,421,421

data_path = "../data/darcy_2d/piececonst_r421_N1024_smooth1"
data1 = loadmat(data_path)

data_in = data1["coeff"]
data_out = data1["sol"]
# data_in = np.array([np.rot90(data_out[i,:,:], k=-1) for i in range(data_out.shape[0])])
# data_out = np.array([np.rot90(data_in[i,:,:], k=-1) for i in range(data_in.shape[0])])
print("data_in.shape:" , data_in.shape)
print("data_out.shape", data_out.shape)

Np_ref = data_in.shape[1]
grid_1d = np.linspace(0, L, Np_ref)
grid_x, grid_y = np.meshgrid(grid_1d, grid_1d)

data_in_ds = data_in[0:n_train, 0::downsample_ratio, 0::downsample_ratio]
grid_x_ds = grid_x[0::downsample_ratio, 0::downsample_ratio]
grid_y_ds = grid_y[0::downsample_ratio, 0::downsample_ratio]
data_out_ds = data_out[0:n_train, 0::downsample_ratio, 0::downsample_ratio]

# x_train, y_train are [n_data, n_x, n_channel] arrays
x_train = torch.from_numpy(
    np.stack(
        (
            data_in_ds,
            np.tile(grid_x_ds, (n_train, 1, 1)),
            np.tile(grid_y_ds, (n_train, 1, 1)),
        ),
        axis=-1,
    ).astype(np.float32)
)
y_train = torch.from_numpy(data_out_ds[:, :, :, np.newaxis].astype(np.float32))
# x_test, y_test are [n_data, n_x, n_channel] arrays
x_test = torch.from_numpy(
    np.stack(
        (
            data_in[-n_test:, 0::downsample_ratio, 0::downsample_ratio],
            np.tile(grid_x[0::downsample_ratio, 0::downsample_ratio], (n_test, 1, 1)),
            np.tile(grid_y[0::downsample_ratio, 0::downsample_ratio], (n_test, 1, 1)),
        ),
        axis=-1,
    ).astype(np.float32)
)
y_test = torch.from_numpy(
    data_out[-n_test:, 0::downsample_ratio, 0::downsample_ratio, np.newaxis].astype(
        np.float32
    )
)

x_train = x_train.reshape(x_train.shape[0], -1, x_train.shape[-1])   # shape: 800,11236,3  (11236 = 106*106 , 106-1 = (421-1) /4)
x_test = x_test.reshape(x_test.shape[0], -1, x_test.shape[-1])
y_train = y_train.reshape(y_train.shape[0], -1, y_train.shape[-1])   # shape: 800,11236,1
y_test = y_test.reshape(y_test.shape[0], -1, y_test.shape[-1])
print("x_train.shape: ",x_train.shape)
print("y_train.shape: ",y_train.shape)




####################################
#compute pca bases
####################################
k_max = 512
Np = (Np_ref + downsample_ratio - 1) // downsample_ratio
pca_data_in = data_in_ds.reshape((data_in_ds.shape[0], -1))[:800,:]
pca_data_out = data_out_ds.reshape((data_out_ds.shape[0], -1))[:800,:]
# if config_model["pca_include_input"]:
#     pca_data = np.vstack(
#         (pca_data, data_in_ds.reshape((data_in_ds.shape[0], -1)))
#     )
# if config_model["pca_include_grid"]:
#     n_grid = 1
#     pca_data = np.vstack((pca_data, np.tile(grid_x_ds, (n_grid, 1))))
#     pca_data = np.vstack((pca_data, np.tile(grid_y_ds, (n_grid, 1))))

# percentage = 0.1
# mask1 = torch.rand(pca_data_in.shape) > percentage
# mask2 = torch.rand(pca_data_out.shape) > percentage
# pca_data_in = (torch.from_numpy(pca_data_in)*mask1).numpy()
# pca_data_out = (torch.from_numpy(pca_data_out)*mask2).numpy()


print("Start SVD with data shape: ", pca_data_out.shape, flush = True)

bases_pca_in, wbases_pca_in = compute_2dpca_bases(Np , k_max , L,  pca_data_in)
bases_pca_in, wbases_pca_in = bases_pca_in.to(device), wbases_pca_in.to(device)

bases_pca_out, wbases_pca_out = compute_2dpca_bases(Np , k_max , L,  pca_data_out)
bases_pca_out, wbases_pca_out = bases_pca_out.to(device), wbases_pca_out.to(device)

data_in.shape: (1024, 421, 421)
data_out.shape (1024, 421, 421)
x_train.shape:  torch.Size([800, 961, 3])
y_train.shape:  torch.Size([800, 961, 1])
Start SVD with data shape:  (800, 961)


In [53]:
%matplotlib inline
from models.losses import LpLoss
from models import  FNN_train, compute_2dFourier_bases, compute_2dpca_bases, UnitGaussianNormalizer
x_normalizer = UnitGaussianNormalizer(x_train, dim=[])
y_normalizer = UnitGaussianNormalizer(y_train, dim=[])
n=10
myloss = LpLoss(d=1, p=2, size_average=False)
# fig, axs = plt.subplots(n,2, figsize=(4*2,4*n))
# mygrid = np.linspace(0, L, Np)
# grid_x, grid_y = np.meshgrid(mygrid, mygrid)
for i in range(n):
    x = x_test[i,:,0].to(device)
    # x1 = x_normalizer.encode(x)
    x1=x
    x_hat = torch.einsum('x,xk -> k',x1,wbases_pca_in)
    x0 = torch.einsum('k,xk -> x',x_hat,bases_pca_in)
    # x0 = x_normalizer.decode(x0)
    print('test',torch.norm(x0-x).item()/torch.norm(x).item())
    # print('xtest',myloss(x,x0).item())
    # # im=axs[i,0].pcolormesh(grid_x, grid_y, x.to('cpu').reshape(Np,Np))
    # # fig.colorbar(im, ax=axs[i,0])
    # # im=axs[i,1].pcolormesh(grid_x, grid_y, x0.to('cpu').reshape(Np,Np))
    # # fig.colorbar(im, ax=axs[i,1])

    x = x_train[i,:,0].to(device)
    # x1 = x_normalizer.encode(x)
    x1=x
    x_hat = torch.einsum('x,xk -> k',x1,wbases_pca_in)
    x0 = torch.einsum('k,xk -> x',x_hat,bases_pca_in)
    # x0 = x_normalizer.decode(x0)
    print('train',torch.norm(x0-x).item()/torch.norm(x).item())
    # print('xtrain',myloss(x,x0).item())

    # x = y_test[i,:,0].to(device)
    # # x1 = y_normalizer.encode(x)
    # x1=x
    # x_hat = torch.einsum('x,xk -> k',x1,wbases_pca_out)
    # x0 = torch.einsum('k,xk -> x',x_hat,bases_pca_out)
    # # x0 = y_normalizer.decode(x0)
    # print('ytest',torch.norm(x0-x).item()/torch.norm(x).item())
    # x = x.unsqueeze(0)
    # x0 = x0.unsqueeze(0)
    # loss = myloss(x,x0)
    # print('test',loss.item())
    # im=axs[i,0].pcolormesh(grid_x, grid_y, x.to('cpu').reshape(Np,Np))
    # fig.colorbar(im, ax=axs[i,0])
    # im=axs[i,1].pcolormesh(grid_x, grid_y, x0.to('cpu').reshape(Np,Np))
    # fig.colorbar(im, ax=axs[i,1])
    # x = y_train[i,:,0].to(device)
    # # x1 = y_normalizer.encode(x)
    # x1=x
    # x_hat = torch.einsum('x,xk -> k',x1,wbases_pca_out)
    # x0 = torch.einsum('k,xk -> x',x_hat,bases_pca_out)
    # # x0 = y_normalizer.decode(x0)
    # # print('ytrain',torch.norm(x0-x).item()/torch.norm(x).item())
    # print('train',myloss(x,x0).item())



test 0.12202412657504426
train 0.042574322955539555
test 0.10809670493162908
train 0.03811437766318393
test 0.08922759853271657
train 0.04113408883660831
test 0.1217855504237831
train 0.03779154900813395
test 0.11049039876736404
train 0.04703398522896091
test 0.1070383313392972
train 0.049982606421776384
test 0.1107767581643044
train 0.04795972416909893
test 0.13861489552493955
train 0.03869884652551351
test 0.1274053949622127
train 0.026094976996212455
test 0.12279163722298808
train 0.03668960453934911
